In [ ]:
#PROBLEM 1
#load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing


#load data
derm_df = pd.read_csv('dermatology.csv', delimiter = '\t')

#correct column names
derm_df = derm_df.rename(columns = {"Family History":"Family Hostory"})
derm_df = derm_df.rename(columns = {"Disappearance":"Disapperance"});

#remove rows with '?'/missing value
derm_df.drop(derm_df.index[derm_df['Age'] == '?'], inplace=True)

#convert columns to numeric
derm_df = derm_df.apply(pd.to_numeric)

#preview data
derm_df.head()

In [2]:
!pip install sklearn

zsh:1: command not found: pip


In [ ]:
#isolating independent feature of Age
x = derm_df['Age']

#isolating class label of Disease
y = derm_df['Disease']

In [ ]:
lr_model = sm.OLS(y,x).fit()
print(lr_model.summary())

#MODEL 1: GRADIENT DESCENT
def gradient_descent(x, y):
    m1 = 0
    c1 = 0
    epochs = 5000 #define number of iterations
    n = len(x)
    L = 0.000643 #learning rate

    for i in range(epochs):
        y_Pred = m1 * x + c1
        cost = (1/n) * sum([val**2 for val in (y-y_Pred)])
        md = -(2/n)*sum(x*(y-y_Pred)) #m derivative
        cd = -(2/n)*sum(y-y_Pred) #c derivative
        m1 = m1 - L * md
        c1 = c1 - L * cd
        print ("m {}, c {}, cost {} iteration {}".format(m1, c1, cost, i))

gradient_descent(x, y)

#To minimize the cost function and find the function minimum, I used 5000 iterations with a learning rate of 0.000643.

#For derm_df, the optimal intercept is 2.3224138138359405 and the optimal gradient is 0.006776863593425256
#this yields the lowest cost function at 2.67

In [ ]:
#MODEL 2: RANDOM FOREST

#isolating independent features(clinical and histopathological attributes)
X = derm_df.iloc[0:, 0:33]

#isolating class label
y = derm_df.iloc[0:, 34]

# normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
X = StandardScaler().fit_transform(X)

rfc = RandomForestClassifier(n_estimators = 100)
rfc.fit(X, y.values.ravel())

predictions = rfc.predict(X)
print("RF ACCURACY:", accuracy_score(y, predictions))
print(confusion_matrix(y, predictions))

#specified number of trees (n_estimators) as 100.

In [ ]:
#MODEL 3: KNN

#isolating independent features(clinical and histopathological attributes)
X = derm_df.iloc[0:, 0:33]
#isolating class label
y = derm_df.iloc[0:, 34]

# normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
X = StandardScaler().fit_transform(X)

#create KNN Classifier model
knn = KNeighborsClassifier(n_neighbors = 9)

#fit  data
knn.fit(X, np.ravel(y))

#predicting on data
predictions = knn.predict(X)

#printing Confusion matrix and accuracy scores
print('KNN confusion matrix on test data')
print(confusion_matrix(y, predictions))

print('Accuracy on test data')
print(accuracy_score(y, predictions))

#To decide k value, I used general rule of thumb of k = sqrt(N)/2. This data set is a bit larger than what we had
#used in previous assignments (N=358). Based off this, I selected 9 as the k value. 

#Accuracy score is 96.65%

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as sch

#MODEL 4: Divisive clustering, K-Means

#isolating independent features
X = derm_df.iloc[0:, 0:33]

# normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
X = StandardScaler().fit_transform(X)

#will use 6 clusters since there are 6 disease types

#K-Means
#scale data
scaler = StandardScaler()
scaler.fit_transform(X)

kmeans = KMeans(n_clusters = 6)
y_means = kmeans.fit_predict(X)

centroids = kmeans.cluster_centers_
#print values for each of the 6 centroids
print(centroids)
print('KMeans', y_means)

In [ ]:
#Model 5: Agglomerative clustering

plt.figure(figsize = (10, 10))
plt.title('Agglomerative clustering')
#Ward
Dendrogram = sch.dendrogram((sch.linkage(X, method = 'ward')))

#will use 6 classes since there are 6 disease types

ac = AgglomerativeClustering(n_clusters = 6)
y_ac = ac.fit_predict(X)

print('Agglomerative clustering', y_ac)

In [ ]:
#PROBLEM 2
#load data
hc_df = pd.read_csv('hatecrime.csv')

#preview data
hc_df.head()

In [ ]:
# missing values
print(hc_df.isna().sum())
#share_non_citizen(3), hate_crimes_per_100k_fbi(5), avg_hate_crimes_per_100k_fbi(1)

In [ ]:
#remove rows with missing values
hc_df = hc_df.dropna()

# normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
X = StandardScaler().fit_transform(X)

In [ ]:
#1. How does income inequality relate to the number of hate crimes and hate incidents? [5 points]

#using linear regression because hate crime label is continuous value
#isolating class label
y = hc_df['hate_crimes_per_100k_splc']
#isolating independent feature of Gini index score
x = hc_df[['gini_index']]
x = sm.add_constant(x)
hc_model = sm.OLS(y, x).fit()
print(hc_model.summary())

#repeating but with avg_hatecrimes_per_100k_fbi label
y = hc_df['avg_hatecrimes_per_100k_fbi']
#isolating independent feature of Gini index score
x = hc_df[['gini_index']]
x = sm.add_constant(x)
hc_model = sm.OLS(y, x).fit()
print(hc_model.summary())

#when comparing gini_index to hate_crimes_per_100k_splc and avg_hatecrimes_per_100k_fbi labels, p-value is > 0.05
#there is a statistically significant relationship between Gini index score and crime
#based off coefficient values, Gini index score and crime rate have a positive relationship
#higher income inequality leatds to higher number of hate crimes and incidents

#linear equation using gini_index to predict hate_crimes_per_100k_splc
#hate_crimes_per_100k_splc = -1.2754  + 3.1512*x

#linear equation using gini_index to predict avg_hatecrimes_per_100k_fbi
#avg_hatecrimes_per_100k_fbi = -1.5307  + 3.7675*x


In [ ]:
#2. How can we predict the number of hate crimes and hate incidents from race/nature of the population? 
#scale labels

#using linear regression because hate crime label is continuous value
#isolating class label
y = hc_df['hate_crimes_per_100k_splc']
#omitting Gini index score
X = hc_df[['median_household_income', 'share_unemployed_seasonal', 'share_population_in_metro_areas', 'share_population_with_high_school_degree', 'share_non_citizen', 'share_white_poverty', 'share_non_white', 'share_voters_voted_trump']]
X = sm.add_constant(X)
hc_model = sm.OLS(y, X).fit()
print(hc_model.summary())
#top 3 lowest p-values: share_voters_voted_trump, share_unemployed_seasonal, median_household_income

#see how this compares to avg_hatecrimes_per_100k_fbi
y = hc_df['avg_hatecrimes_per_100k_fbi']
#omitting Gini index score
X = hc_df[['median_household_income', 'share_unemployed_seasonal', 'share_population_in_metro_areas', 'share_population_with_high_school_degree', 'share_non_citizen', 'share_white_poverty', 'share_non_white', 'share_voters_voted_trump']]
X = sm.add_constant(X)
hc_model = sm.OLS(y, X).fit()
print(hc_model.summary())
#top 3 lowest p-values: share_voters_voted_trump, share_unemployed_seasonal, share_non_citizen


#linear equation using top 3 variables stated above (in listed order) to predict hate_crimes_per_100k_splc
#hate_crimes_per_100k_splc = -0.0038 - (1.3202*x1) + (3.5613*x2) - (4.89e-06*x3)

#linear equation using top 3 variables stated above to predict avg_hatecrimes_per_100k_fbi
#avg_hatecrimes_per_100k_fbi = 0.4265 - (-1.3202*x1) + (3.5613*x2) + (0.3391*x3)

In [ ]:
#3. How does the number of hate crimes vary across states? Is there any similarity in number of hate incidents (per 100,000 people) between some states than in others — both according to the SPLC after the election and the FBI before it?
#isolating class label

#add number column to refer to states
hc_df['State_Num'] = np.arange(len(hc_df))

#Clustering using K Means

#isolate and scale attributes
X = hc_df[['hate_crimes_per_100k_splc', 'avg_hatecrimes_per_100k_fbi']]
#scale data
scaler = StandardScaler()
scaler.fit_transform(X)

#K-Means
kmeans = KMeans(n_clusters = 3)
y_means = kmeans.fit_predict(X)

centroids = kmeans.cluster_centers_
#print values for each of the 3 centroids
print(centroids)
print('KMeans', y_means)

#visualization
plt.figure(figsize = (10,10))
plt.title('Divisive Clustering Using K-Means')
#visualizing 
plt.scatter(X['hate_crimes_per_100k_splc'], X['avg_hatecrimes_per_100k_fbi'], c = y_means, cmap = 'rainbow')
plt.scatter(centroids[:,0], centroids[:,1], c = 'black', s = 100)
plt.show()

#Washington DC is shown to have the highest crime rate and incident (cluster 2). 

#Colorado, Conneticut, Kansas, Maine, Maryland, Michigan, Nevada, West Virginia follow after (cluster 2)